In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict

In [25]:
df = pd.DataFrame()
for chunk in pd.read_csv('./news_dataset.csv', usecols=['content'], chunksize=1000):
    df = pd.concat([df, chunk], ignore_index=True)

In [3]:
df.head()

,content
0,WASHINGTON — Congressional Republicans have...
1,"After the bullet shells get counted, the blood..."
2,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Death may be the great equalizer, but it isn’t..."
4,"SEOUL, South Korea — North Korea’s leader, ..."


In [30]:
import re
df["content"] = df["content"].str.lower() 

# df["content"] = df["content"].str.translate(None, ",!.; -@!%^&*)(\\\'\"")
df['content'] = df['content'].map(lambda x: re.sub(r'[,!.;+-@!%^&*)(\\\'\"“”’—]', '', str(x)))
df['content'] = df['content'].map(lambda x: re.sub(r'\W+', ' ', str(x)))

In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words("english")
df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Quentindh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [34]:
df["content"]


0         washington congressional republicans new fear ...
1         bullet shells get counted blood dries votive c...
2         walt disneys bambi opened critics praised spar...
3         death may great equalizer isnt necessarily eve...
4         seoul south korea north koreas leader kim said...
                                ...                        
141580    uber driver keith avila picked passenger looke...
141581    crews friday continued search lake erie plane ...
141582    obama administration announced massive effort ...
141583    story updated new law michigan prohibit local ...
141584    nations first recreational marijuana shop open...
Name: content, Length: 141585, dtype: object

In [35]:
nr_topics = 10
topics = list(range(nr_topics))
topics

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [36]:
doc_topics = []
word_topics = defaultdict(lambda: defaultdict(int))

assigned = []
for index, row in df.iterrows():
    if index%1000 == 0:
        print(index)
        if index == 2000:
            break
    t = defaultdict(int)
    assigned_doc = []
    for word in row["content"].split():
        word_topic = random.sample(topics, 1)[0]
        word_topics[word][word_topic] += 1
        assigned_doc.append(word_topic)
        t[word_topic] += 1
    assigned.append(assigned_doc)
    doc_topics.append(t)
        

0
1000
2000


In [7]:
doc_topics

[defaultdict(int,
             {8: 78,
              4: 81,
              5: 98,
              6: 89,
              1: 82,
              7: 98,
              0: 95,
              3: 76,
              2: 110,
              9: 77}),
 defaultdict(int,
             {0: 484,
              5: 486,
              2: 481,
              9: 482,
              6: 471,
              8: 515,
              1: 494,
              7: 477,
              3: 435,
              4: 445}),
 defaultdict(int,
             {4: 262,
              2: 257,
              1: 213,
              6: 256,
              8: 235,
              5: 232,
              7: 237,
              3: 260,
              0: 221,
              9: 216}),
 defaultdict(int,
             {3: 242,
              9: 210,
              7: 231,
              2: 196,
              5: 198,
              1: 218,
              6: 216,
              0: 200,
              8: 208,
              4: 200}),
 defaultdict(int,
             {3: 67,
          

In [ ]:
word_topics

In [ ]:
assigned[0]

In [ ]:
alpha = 10
gamma = 4

def sum_doc(doc_index):
    return len(assigned[doc_index]) + nr_topics*alpha

def sum_topic(topic):
    s = 0
    for doc in doc_topics:
        s += doc[topic] + gamma
        
    return s

for index, row in df.iterrows():
    print(index)
    for i, word in enumerate(row["content"].split()):
        t = assigned[index][i]
        word_topics[word][t] -= 1
        doc_topics[index][t] -= 1
        
        topic_scores = []
        for top in topics:
            n_dk = doc_topics[index][t]
            v_kwdn = word_topics[word][t]

            s1 = sum_doc(index)
            s2 = sum_topic(t)
            topic_scores.append(((n_dk + alpha)/s1) * ((v_kwdn + gamma)/s2))
        
        z_new = random.choices(topics, weights=topic_scores, k=1)[0]
        word_topics[word][z_new] += 1
        doc_topics[index][z_new] += 1
        assigned[index][i] = z_new
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
